Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: Policy Search

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The player **taking the last object wins**.

* Task3.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task3.2: An agent using evolved rules
* Task3.3: An agent using minmax
* Task3.4: An agent using reinforcement learning

## Instructions

* Create the directory `lab3` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.

## Deadlines ([AoE](https://en.wikipedia.org/wiki/Anywhere_on_Earth))

* Sunday, December 4th for Task3.1 and Task3.2
* Sunday, December 11th for Task3.3 and Task3.4
* Sunday, December 18th for all reviews

In [89]:
import logging
from collections import namedtuple
import random
from typing import Callable
from copy import deepcopy
from itertools import accumulate, chain
from operator import xor

## The *Nim* and *Nimply* classes

In [90]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [91]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

## Sample (and silly) startegies 

In [92]:
def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [93]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

# My strategies

In [228]:
def active_rows_number(state: Nim) -> int: 
    return sum(o > 0 for o in state.rows)
    
def pick_max_from_lowest(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

def pick_max_from_highest(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the highest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (m[0], m[1])))

def pick_min_from_lowest(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*min(possible_moves, key=lambda m: (m[0], m[1])))

def pick_min_from_highest(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (m[0], -m[1])))

def count_rows_and_choose(state: Nim) -> Nimply:
    rows = active_rows_number(state)
    if rows % 2 == 0:
        return pick_min_from_highest(state)
    else:
        return pick_max_from_highest(state)

def pick_odd_number_of_elements(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    ns = [n for n in range(1, state.rows[row]+1) if n%2!=0]
    print(ns)
    num_objects = random.choice(ns)
    return Nimply(row, num_objects)

def pick_even_number_of_elements(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 1])
    ns = [n for n in range(1, state.rows[row]+1) if n%2==0]
    print(ns)
    num_objects = random.choice(ns)
    return Nimply(row, num_objects)


In [254]:
nim = Nim(5)
print(nim)
pick_even_number_of_elements(nim)


<1 3 5 7 9>
[2, 4, 6, 8]


Nimply(row=4, num_objects=6)

In [95]:
def nim_sum(state: Nim) -> int:
    *_, result = accumulate(state.rows, xor)
    return result

def active_rows_number(state: Nim) -> int: 
    return sum(o > 0 for o in state.rows)

def cook_status(state: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    cooked["active_rows_number"] = active_rows_number(state) #sum(o > 0 for o in state.rows)
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["nim_sum"] = nim_sum(state)

    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp), active_rows_number(tmp)))
    cooked["brute_force"] = brute_force

    return cooked

In [96]:
def optimal_startegy(state: Nim) -> Nimply:
    data = cook_status(state)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]

In [97]:
def andrea(state: Nim) -> Nimply:
    data = cook_status(state)
    moves = (bf for bf in data["brute_force"] if bf[1] == 0)
    moves = chain(moves, (bf for bf in data["brute_force"] if bf[2] > 1 or bf[2] % 2 == 0 or bf[1] % 2 == 0))
    return next(moves, random.choice(data["brute_force"]))[0]

In [98]:
def make_strategy(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)

        if random.random() < genome["p"]:
            ply = Nimply(data["shortest_row"], random.randint(1, state.rows[data["shortest_row"]]))
        else:
            ply = Nimply(data["longest_row"], random.randint(1, state.rows[data["longest_row"]]))

        return ply

    return evolvable

In [99]:
def make_my_strategy(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:

        if random.random() < genome["p"]:
            ply = count_rows_and_choose(state)
        else:
            if random.random() < genome["alfa"]:
                # pick max
                if random.random() < genome["beta"]:
                    # pick from highest
                    ply = pick_max_from_highest(state)
                else:
                    ply = pick_max_from_lowest(state)
            else:
                # pick min
                if random.random() < genome["beta"]:
                    # pick from highest
                    ply = pick_min_from_highest(state)
                else:
                    ply = pick_min_from_lowest(state)

        return ply

    return evolvable

In [120]:
NUM_MATCHES = 100
NIM_SIZE = 11


def evaluate(strategy: Callable) -> float:
    opponent = (strategy, pure_random)
    # opponent = (pure_random, strategy)
    won = 0

    for m in range(NUM_MATCHES):
        nim = Nim(NIM_SIZE)
        player = 0
        while nim:
            ply = opponent[player](nim)
            nim.nimming(ply)
            player = 1 - player
        if player == 1:
            won += 1
    return won / NUM_MATCHES

# Try to evolve
## Setting parameters

In [111]:
Individual = namedtuple("Individual",["genome", "fitness"])
POPULATION_SIZE = 10
NUM_GENERATIONS = 100
OFFSPRING_SIZE = 5
MUT_RATE = 0.3

def compute_fitness(genome):
    return evaluate(make_my_strategy(genome))

def tournament(population, tournament_size=2): 
    return max(random.choices(population, k=tournament_size), key=lambda i: i.fitness) 

def mutation(g):
    mutate_what = random.random()
    if mutate_what < 0.3:
        g_mut = {"p": random.random(), "alfa": g["alfa"], "beta": g["beta"]}
    elif mutate_what < 0.6:
        g_mut = {"p": g["p"], "alfa": random.random(), "beta": g["beta"]}
    else:
        g_mut = {"p": g["p"], "alfa": g["alfa"], "beta": random.random()}
    return g_mut

def crossover(g1, g2):
    if random.random() < 0.5:
        p = g1["p"]
    else: 
        p = g2["p"]
    if random.random() < 0.5:
        alfa = g1["alfa"]
    else: 
        alfa = g2["alfa"]
    if random.random() < 0.5:
        beta = g1["beta"]
    else: 
        beta = g2["beta"]
    g_crossover = {"p": p, "alfa": alfa, "beta": beta}
    return g_crossover   

## Evolution

In [121]:
def my_genetic_algorithm(population):
    for generation in range(NUM_GENERATIONS):
        offspring = list()
        for i in range(OFFSPRING_SIZE):
            if random.random() < MUT_RATE:
                p = tournament(population)
                o = mutation(p.genome)
            else:
                p1 = tournament(population)                 # promising genome 1
                p2 = tournament(population)                 # promising genome 2
                o = crossover(p1.genome, p2.genome)
            f = compute_fitness(o)
            offspring.append(Individual(o,f))

        population += offspring
        population = sorted(population, key = lambda i: i.fitness, reverse=True)[:POPULATION_SIZE]

        best_so_far = population[0]
        if(generation % 10 == 0):
            print(f"GEN #{generation}\tGENOME: {best_so_far.genome}\tFITNESS: {best_so_far.fitness}")

In [122]:
# a genome is composed by three fields: p, alfa, beta
# population composed by randomic values

population = list()
for _ in range(POPULATION_SIZE):
    p = random.random()
    alfa = random.random()
    beta = random.random()
    genome = {"p": p, "alfa": alfa, "beta": beta}
    population.append(Individual(genome, compute_fitness(genome)))

my_genetic_algorithm(population)

GEN #0	GENOME: {'p': 0.8264625109786478, 'alfa': 0.9254463205964043, 'beta': 0.25786563093822856}	FITNESS: 0.86
GEN #10	GENOME: {'p': 0.9387797605547467, 'alfa': 0.9254463205964043, 'beta': 0.25786563093822856}	FITNESS: 0.9
GEN #20	GENOME: {'p': 0.9387797605547467, 'alfa': 0.9254463205964043, 'beta': 0.25786563093822856}	FITNESS: 0.9
GEN #30	GENOME: {'p': 0.9387797605547467, 'alfa': 0.9254463205964043, 'beta': 0.25786563093822856}	FITNESS: 0.92
GEN #40	GENOME: {'p': 0.9387797605547467, 'alfa': 0.9254463205964043, 'beta': 0.25786563093822856}	FITNESS: 0.92
GEN #50	GENOME: {'p': 0.9387797605547467, 'alfa': 0.9254463205964043, 'beta': 0.25786563093822856}	FITNESS: 0.92
GEN #60	GENOME: {'p': 0.9387797605547467, 'alfa': 0.9254463205964043, 'beta': 0.3667943256779338}	FITNESS: 0.93
GEN #70	GENOME: {'p': 0.9387797605547467, 'alfa': 0.9254463205964043, 'beta': 0.3667943256779338}	FITNESS: 0.93
GEN #80	GENOME: {'p': 0.9387797605547467, 'alfa': 0.9254463205964043, 'beta': 0.9344268473905292}	FIT

## Oversimplified match

In [123]:
logging.getLogger().setLevel(logging.DEBUG)

strategy = (count_rows_and_choose, pick_min_from_highest)

nim = Nim(5)
logging.debug(f"status: Initial board  -> {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")

DEBUG:root:status: Initial board  -> <1 3 5 7 9>
DEBUG:root:status: After player 0 -> <1 3 5 7 0>
DEBUG:root:status: After player 1 -> <1 3 5 6 0>
DEBUG:root:status: After player 0 -> <1 3 5 5 0>
DEBUG:root:status: After player 1 -> <1 3 5 4 0>
DEBUG:root:status: After player 0 -> <1 3 5 3 0>
DEBUG:root:status: After player 1 -> <1 3 5 2 0>
DEBUG:root:status: After player 0 -> <1 3 5 1 0>
DEBUG:root:status: After player 1 -> <1 3 5 0 0>
DEBUG:root:status: After player 0 -> <1 3 0 0 0>
DEBUG:root:status: After player 1 -> <1 2 0 0 0>
DEBUG:root:status: After player 0 -> <1 1 0 0 0>
DEBUG:root:status: After player 1 -> <1 0 0 0 0>
DEBUG:root:status: After player 0 -> <0 0 0 0 0>
INFO:root:status: Player 0 won!
